In [6]:
import pandas as pd
import sqlite3

In [14]:
#Load data
df = pd.read_csv('data/data_google_maps.csv')

df.head()

,names,summary,cusine,price,reviews_count,rating,address,url
0,Restaurant Story,Chef Tom Sellers' novel approach to British cl...,Fine dining restaurant,££££,418 reviews,4.6,"199 Tooley St, London SE1 2JX",https://www.google.com/maps/place/Restaurant+S...
1,Galvin La Chapelle,NaN,French restaurant,££££,"1,901 reviews",4.6,NaN,https://www.google.com/maps/place/Galvin+La+Ch...
2,City Social,NaN,Modern European restaurant,££££,"1,187 reviews",4.5,NaN,https://www.google.com/maps/place/City+Social/...
3,Brawn,Mediterranean small plates menu with an 'all t...,Mediterranean restaurant,££,615 reviews,4.7,"49 Columbia Rd, London E2 7RG",https://www.google.com/maps/place/Brawn/@51.52...
4,Il Bordello,Old-school Italian restaurant with an authenti...,Italian restaurant,££,"1,170 reviews",4.5,"metropolitan wharf, 70 Wapping Wall, London E1...",https://www.google.com/maps/place/Il+Bordello/...


In [12]:
#Remove duplicates, if any 
df.drop_duplicates()

#Remove rows where cusine = "Not a restaurant"
df = df[df.cusine != 'Not a restaurant']

#Remove rows where there are no ratings 
df = df[df.rating != 'Not Available']


In [13]:
len(df)

373